# HEREDITARY Architecture Playground


**NOTE**: this notebook is written in JavaScript, so you need Jupyter + a JavaScript interpreter to run it. You may use the Deno interpreter, which is powerful and comes with a prebuild support for notebooks
```bash
curl -fsSL https://deno.land/install.sh | sh  # Install Deno
"$HOME/.deno/bin/deno" jupyter --install      # Add Deno to Jupyter
```
now just restart VSCode/Jupyter and you should see Deno amongst the kernels

In [8]:
const hello = "hello world!"
console.log(hello)

hello world!


## Dremio API Playground

<https://docs.dremio.com/cloud/reference/api/>

In [1]:
const hostname = "http://localhost:9047";
const endpoint = `${hostname}/api/v3`;
const userName = 'dremioUser'
const password = 'dremioPass'

const post = async (url: string, data: Record<string, unknown>) => {
    const response = await fetch(url, {
        method: "POST",
        headers: {
        "Content-Type": "application/json",
        },
        body: JSON.stringify(data),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);  
    return response.json();
}


// create first user if none
const firstName = 'admin'
const lastName = ''
const email = 'luca.fabbian.1999@gmail.com'
const createdAt = Date.now()
const response = await fetch(`${hostname}/apiv2/bootstrap/firstuser`, {
    method: "PUT",
    headers: {
      "Authorization": '_dremionull',
      "Content-Type": "application/json",
    },
    body: JSON.stringify({userName, password, firstName, lastName, email, createdAt}),
});

if (!response.ok ){
  if(response.status !== 400) throw new Error(`Unexpected HTTP error! status: ${response.status}`);
  console.log("User alredy there, no need to create a new one")
}else{
  console.log("New user created", await response.json())
}


// get auth token
const {token, expires } = await post(`${hostname}/apiv2/login`, { userName, password })
console.log('Auth token:', token, '\nExpires at:', (new Date(expires)).toLocaleTimeString('en-GB'), '(30hours)')

// helper function to make dremio api calls
const dremio = async(method: 'GET'|'POST', url: string, data: Record<string, unknown>)  => {
    const response = await fetch(`${endpoint}${url}`, {
        method,
        headers: {
            'Authorization': `Bearer ${token}`,
            "Content-Type": "application/json",
        },
        body: JSON.stringify(data),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);  
    return response.json();
}

User alredy there, no need to create a new one
Auth token: cklb0qhftbp2o24kgop5vm6532 
Expires at: 16:44:47 (30hours)


In [17]:
// inspect source(s)
await dremio('GET', '/catalog/e3dcfd81-19cd-4957-bd5c-ad7f87723a1a')

{
  entityType: "source",
  config: {
    path: "/csv",
    defaultCtasFormat: "ICEBERG",
    isPartitionInferenceEnabled: false
  },
  state: { status: "good", suggestedUserAction: "", messages: [] },
  id: "e3dcfd81-19cd-4957-bd5c-ad7f87723a1a",
  tag: "/QhfsScTZGU=",
  type: "NAS",
  name: "mockcsv",
  createdAt: "1970-01-01T00:00:00.000Z",
  metadataPolicy: {
    authTTLMs: 86400000,
    namesRefreshMs: 3600000,
    datasetRefreshAfterMs: 3600000,
    datasetExpireAfterMs: 10800000,
    datasetUpdateMode: "PREFETCH_QUERIED",
    deleteUnavailableDatasets: true,
    autoPromoteDatasets: false
  },
  accelerationGracePeriodMs: 10800000,
  accelerationRefreshPeriodMs: 3600000,
  accelerationRefreshSchedule: "0 0 8 * * *",
  accelerationActivePolicyType: "PERIOD",
  accelerationNeverExpire: false,
  accelerationNeverRefresh: false,
  children: [
    {
      id: "dremio:/mockcsv/email.csv",
      path: [ "mockcsv", "email.csv" ],
      type: "FILE"
    },
    {
      id: "315e799c-f634-

In [11]:
await dremio('POST', '/catalog', {
    entityType: 'source',
    type: "POSTGRES",
    name: "mockdb",
    config: {
        path: "/csv",
        defaultCtasFormat: "ICEBERG",
        isPartitionInferenceEnabled: false,
    },
})

{
  entityType: "source",
  config: {
    hostname: "db",
    port: "5432",
    databaseName: "example",
    username: "user",
    password: "$DREMIO_EXISTING_VALUE$",
    authenticationType: "MASTER",
    fetchSize: 200,
    useSsl: false,
    encryptionValidationMode: "CERTIFICATE_AND_HOSTNAME_VALIDATION",
    maxIdleConns: 8,
    idleTimeSec: 60,
    queryTimeoutSec: 0
  },
  state: { status: "good", suggestedUserAction: "", messages: [] },
  id: "b36862a4-7cb6-4863-85fb-9e42e81904e2",
  tag: "owPh6USGxjE=",
  type: "POSTGRES",
  name: "mockdb2",
  createdAt: "2024-09-04T09:00:24.756Z",
  metadataPolicy: {
    authTTLMs: 86400000,
    namesRefreshMs: 3600000,
    datasetRefreshAfterMs: 3600000,
    datasetExpireAfterMs: 10800000,
    datasetUpdateMode: "PREFETCH_QUERIED",
    deleteUnavailableDatasets: true,
    autoPromoteDatasets: false
  },
  accelerationGracePeriodMs: 0,
  accelerationRefreshPeriodMs: 0,
  accelerationRefreshSchedule: "0 0 8 * * ?",
  accelerationActivePolicyTyp

In [ ]:
await dremio('POST', '/catalog', {
    entityType: 'source',
    type: "POSTGRES",
    name: "mockdb",
    config: {
        hostname: "db",
        port: "5432",
        databaseName: "example",
        username: "user",
        password: "pass",
        authenticationType: "MASTER",
        fetchSize: 200,
        useSsl: false,
        encryptionValidationMode: "CERTIFICATE_AND_HOSTNAME_VALIDATION",
        maxIdleConns: 8,
        idleTimeSec: 60,
        queryTimeoutSec: 0
    },
})

## Generate mock data

In [2]:
const seed = 12345;

// Simple seedable random number generator (Linear Congruential Generator)
const seededRandom = (seed: number): () => number  => {
    let value = seed % 2147483647;
    return () => {
        value = (value * 16807) % 2147483647;
        return (value - 1) / 2147483646;
    }
}
let rng = seededRandom(seed);
const randomString = (chars: string, length: number): string => {
    let result = '';
    for (let i = 0; i < length; i++) {
        result += chars.charAt(Math.floor(rng() * chars.length));
    }
    return result;
}
const randomLowercase = (length: number) => randomString('abcdefghijklmnopqrstuvwxyz', length)
const randomName = (length: number) => randomString('ABCDEFGHIJKLMNOPQRSTUVWXYZ', 1) + randomLowercase(length - 1)
const randomElement = <T>(array: T[]) : T => array[Math.floor(rng()*array.length)]


const domains = ['gmail.com', 'cs.aau.dk', 'unipd.it', 'studenti.unipd.it', 'hotmail.com', 'example.com']
const cities = ['Aalborg', 'Padova', 'New york', 'Roma', 'London', 'Paris', 'Venice']


In [1]:
const filePath = './generated_mockdata.csv';
const seed = 12345
rng = seededRandom(seed); // resets the seed
Deno.writeTextFileSync(filePath, 'Name,Surname,Email,Value,City\n', { append: false });

const rowsToGenerate = (1000 * 1000 * 16) * 120; // Number of rows ( 1Gb = 1000*1000*16)
const chunkSize = 10000;        // Chunk size to control memory usage

for (let chunk = 0; chunk < rowsToGenerate / chunkSize; chunk++) {
    let data = '';
    for (let i = 0; i < chunkSize; i++) {
        const name = randomName(8);
        const surname = randomName(10 + Math.floor(rng()*6));
        const domain = randomElement(domains)
        const email = `${name}.${surname}@${domain}`;
        const value = Math.floor(rng() * 10000);
        const city = randomElement(cities)
        data += `${name},${surname},${email},${value},${city}\n`;
    }
    Deno.writeTextFileSync(filePath, data, { append: true });
}

console.log(`Seeded CSV file generated at ${filePath}`);


ReferenceError: seededRandom is not defined

In [3]:
import { TextLineStream } from "jsr:@std/streams@0.223.0/text-line-stream";


const logMatches = async (filename: string) => {
    using f = await Deno.open(filename);
    const readable = f.readable
        .pipeThrough(new TextDecoderStream()) // decode Uint8Array to string
        .pipeThrough(new TextLineStream()) // split string line by line


    for await (const data of readable) {
        const line : string = data
        if(line.startsWith('Esscqwxx,')) console.log(line)
    }
}

In [5]:
await logMatches('./mockdata/csv/generated_mockdata_30GB.csv')

In [4]:
logMatches('./mockdata/csv/generated_mockdata_60GB.csv')

Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova


In [6]:
import { TextLineStream } from "jsr:@std/streams@0.223.0/text-line-stream";
using f = await Deno.open('./generated_mockdata.csv');
const readable = f.readable
  .pipeThrough(new TextDecoderStream()) // decode Uint8Array to string
  .pipeThrough(new TextLineStream()) // split string line by line


for await (const data of readable) {
    const line : string = data
    if(line.startsWith('Esscqwxx,')) console.log(line)
}

Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205,Padova
Esscqwxx,Qyboirdgwujtpjq,Esscqwxx.Qyboirdgwujtpjq@cs.aau.dk,8205